In [9]:
import sys
import torch

print(sys.executable)
print(torch.__file__)
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())


d:\study\Super-Resolution\.venv\Scripts\python.exe
d:\study\Super-Resolution\.venv\Lib\site-packages\torch\__init__.py
2.9.1+cpu
None
False


In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

### Data loading
We will create a custom dataset class for loading our images. This class will inherit from `torch.utils.data.Dataset` and implement the necessary methods to load and preprocess the images (and maybe data augmentation in the future).

In [2]:
class SuperResolutionDataset(Dataset):
    def __init__(self, npz_file_path):
        """Load tensors from .npz file"""
        data = np.load(npz_file_path)
        self.hr_images = torch.from_numpy(data['hr'])
        self.lr_images = torch.from_numpy(data['lr'])

    def __len__(self):
        return len(self.hr_images)

    def __getitem__(self, idx):
        return self.lr_images[idx], self.hr_images[idx]

training_dataset = SuperResolutionDataset('../data/hr_lr_images.npz')
training_dataloader = DataLoader(training_dataset, batch_size=32, shuffle=True)

In [6]:
import sys
sys.path.insert(0, '..')
from src.model import SuperResolutionModel

The line below just a sanitty check to ensure that our model is working as expected. Sometimes notebooks environments are populated.

In [7]:
model = SuperResolutionModel()
x = torch.randn(1, 3, 32, 32)
y = model(x)
print(type(y))
print(y.shape)

<class 'torch.Tensor'>
torch.Size([1, 3, 128, 128])


### Check your model shape and size
The model should not exceed 5 million parameters.

In [8]:
# device refers to the GPU or the CPU, depending whether GPU is available.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model = SuperResolutionModel().to(device)

# display information about the model
summary(model, (3,32,32))

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Trainable parameters:', params)
if params > 5_000_000:
    raise Exception('Your model is unecessarily complex, scale down!')

Using device: cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           4,864
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          36,928
              ReLU-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 48, 32, 32]          27,696
      PixelShuffle-6          [-1, 3, 128, 128]               0
Total params: 69,488
Trainable params: 69,488
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 2.75
Params size (MB): 0.27
Estimated Total Size (MB): 3.03
----------------------------------------------------------------
Trainable parameters: 69488
